data source:
* https://www.kaggle.com/competitions/playground-series-s5e6
* https://www.kaggle.com/datasets/irakozekelly/fertilizer-prediction

# import

In [ ]:
# pip install optuna

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import optuna
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler

from numba import cuda

if cuda.is_available():
  import cupy as cp

RANDOM_STATE = 13

In [ ]:
def mapk(actual, predicted):
    """
    :actual: np.array([[x], [...]])
    :predicted: np.array([[x1, x2, x3, ..., xn], [...]]), indexes of top-N, index equal class label
    """
    matches = (actual == predicted)
    matches_index = np.argmax(matches, axis=1)
    is_hit = matches.any(axis=1)

    return np.mean(np.where(is_hit, 1 / (matches_index+1), 0))


def get_metrics(y, y_pred):
  return log_loss(y, y_pred), mapk(y, np.argsort(y_pred)[:, -3:][:, ::-1])

In [ ]:
train = pd.read_csv('train.csv').drop(columns='id')
original = pd.read_csv('Fertilizer Prediction.csv')
predict = pd.read_csv('test.csv')

cols_object = ['Soil Type', 'Crop Type', 'Fertilizer Name'] # train.select_dtypes('object').columns
target = 'Fertilizer Name'

In [ ]:
# label encoding

label_encoder_values = {}
label_encoder_values_inverse = {}

for col in cols_object:
    unique_values = train[col].unique()
    encode_values = { val: i for i, val in enumerate(unique_values) }

    label_encoder_values[col] = encode_values
    label_encoder_values_inverse[col] = dict(zip(encode_values.values(), encode_values.keys()))

    train[col] = train[col].map(encode_values)
    original[col] = original[col].map(encode_values)

    if col != target:
        predict[col] = predict[col].map(encode_values)

In [ ]:
# dtype to np.float32

for col in train.columns:
    train[col] = train[col].astype(np.float32)
    original[col] = original[col].astype(np.float32)

    if col != target:
        predict[col] = predict[col].astype(np.float32)

In [ ]:
X = train.drop(columns=[target]).values
y = train[[target]].values

X_original = original.drop(columns=[target]).values
y_original = original[[target]].values

X_pred = predict.drop(columns=['id']).values
y_pred = predict[['id']].copy()

# optuna. base models hyperparameters selection

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=y, train_size=0.35)

In [ ]:
def objective(trial, X, y, shuffle=True, random_state=RANDOM_STATE):
    params = dict(
        max_depth=trial.suggest_int("max_depth", 2, 5),
        learning_rate=trial.suggest_float("learning_rate", 0.001, 0.1),
        subsample=trial.suggest_float("subsample", 0.7, 0.95),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.4, 0.7),
        reg_alpha=trial.suggest_float("reg_alpha", 0.0, 3.0),
        reg_lambda=trial.suggest_float("reg_lambda", 0.5, 5.0),
        min_child_weight=trial.suggest_int("min_child_weight", 1, 5),
        n_estimators=7000,
        num_class=7,
        objective='multi:softprob',
        eval_metric='mlogloss',
        booster='gbtree',
        tree_method='hist',
        device='cuda' if cuda.is_available() else 'cpu',
        n_jobs=-1,
        early_stopping_rounds=50,
        random_state=random_state,
    )

    X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=shuffle, stratify=y, test_size=0.3)

    if cuda.is_available(): # CPU -> GPU
      X_train, y_train = cp.asarray(X_train), cp.asarray(y_train)
      X_val, y_val = cp.asarray(X_val), cp.asarray(y_val)

    model = XGBClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=0)
    y_val_proba = model.predict_proba(X_val)

    if cuda.is_available(): # GPU -> CPU
      y_val = y_val.get()

    val_logloss, val_mapk = get_metrics(y_val, y_val_proba)

    trial.set_user_attr("mapk", val_mapk)

    return val_logloss


study = optuna.create_study(directions=['minimize'], sampler=optuna.samplers.TPESampler())
study.optimize(
    lambda trial: objective(trial, X_train, y_train),
    n_trials=50,
    n_jobs=-1,
    show_progress_bar=True
)


study_result = study.trials_dataframe()
study_result_dict = { trial.number: trial.params for trial in study.trials}

In [ ]:
study_result.sort_values(by='value', ascending=True).head(5)

In [ ]:
# study_result_dict[]

In [ ]:
study_result_params = list(study_result.columns[study_result.columns.str.contains('params_')])
study_result_params.append('value')

ss = StandardScaler()
ss_study_result = ss.fit_transform(study_result[study_result_params].values)
ss_study_result = pd.DataFrame(ss_study_result, columns=study_result_params)
ss_study_result = ss_study_result.join(study_result[['number']], rsuffix='_r').sort_values(by='number', ascending=True)

In [ ]:
counter = 0
fig, axes = plt.subplots(len(study_result_params[:-1]), 2, figsize=(20,30))

for param in study_result_params[:-1]:
    if counter == 0:
      axes[counter, 0].set_title('Standard Scaled Data')
      axes[counter, 1].set_title('Original Data')

    sns.lineplot(ax=axes[counter, 0], data=ss_study_result, x='number', y=param, label=param)
    sns.lineplot(ax=axes[counter, 0], data=ss_study_result, x='number', y='value', label='value')
    axes[counter, 0].grid()
    sns.lineplot(ax=axes[counter, 1], data=study_result, x='number', y=param, label=param)
    axes[counter, 1].grid()
    counter = counter + 1

# cross validation score

In [ ]:
def cross_val_score(model, X, y, X_original=None, y_original=None, n_splits=5, shuffle=True, random_state=RANDOM_STATE):
    '''
    designed for xgb classifier
    :model: xgb classifier
    :X, y: np.array
    '''

    logloss_scores, mapk_scores = list(), list()
    kf = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):

        print(f'FOLD {fold} running ...')

        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        if X_original is not None and y_original is not None:
            X_train = np.vstack([X_train, X_original])
            y_train = np.vstack([y_train, y_original])

        if cuda.is_available(): # CPU -> GPU
            X_train, y_train = cp.asarray(X_train), cp.asarray(y_train)
            X_val, y_val = cp.asarray(X_val), cp.asarray(y_val)

        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=1000)
        y_val_pred = model.predict_proba(X_val)

        if cuda.is_available(): # GPU -> CPU
            y_val = y_val.get()

        logloss_score, mapk_score = get_metrics(y_val, y_val_pred)
        logloss_scores.append(logloss_score)
        mapk_scores.append(mapk_score)

        print(f'logloss: {logloss_score}, mapk: {mapk_score}')

    return np.mean(logloss_scores), np.mean(mapk_scores)

In [ ]:
params = {
    # 'max_depth': 8,
    # 'learning_rate': 0.027346170065478962,
    # 'subsample': 0.9379502039625413,
    # 'colsample_bytree': 0.4095558867111752,
    # 'reg_alpha': 1.0357095290998455,
    # 'reg_lambda': 1.7172337694124145,
    # 'min_child_weight': 5,
    'max_depth': 12,
    'learning_rate': 0.02408978898626546,
    'subsample': 0.8722619176156262,
    'colsample_bytree': 0.44757526648072804,
    'reg_alpha': 2.8463847461990315,
    'reg_lambda': 0.6685176738995261,
    'min_child_weight': 10,
    # --
    'n_estimators': 7000,
    'num_class': 7,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'booster': 'gbtree',
    'tree_method': 'hist',
    'device': 'cuda' if cuda.is_available() else 'cpu',
    'n_jobs': -1,
    'early_stopping_rounds': 100,
    'random_state': RANDOM_STATE,
}
model = XGBClassifier(**params)
res = cross_val_score(model=model, X=X, y=y, X_original=X_original, y_original=y_original)

# stacking

In [ ]:
def cross_validation_stacking(models, X, y, X_pred, X_original=None, y_original=None,
                              n_splits=5, shuffle=True, random_state=RANDOM_STATE):

    X_folds, y_folds = list(), list()
    val_preds = { model: list() for model in models }
    pred_preds = { model: np.zeros((X_pred.shape[0], len(np.unique(y)))) for model in models }
    kf = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    if cuda.is_available(): # CPU -> GPU
        X_pred = cp.asarray(X_pred)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):

        print(f'FOLD {fold} running ...')

        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        if X_original is not None and y_original is not None:
            X_train = np.vstack([X_train, X_original])
            y_train = np.vstack([y_train, y_original])

        X_folds.append(X_val)
        y_folds.append(y_val)

        if cuda.is_available(): # CPU -> GPU
            X_train, X_val = cp.asarray(X_train), cp.asarray(X_val)
            y_train, y_val = cp.asarray(y_train), cp.asarray(y_val)

        for model_name, model in models.items():

            print(f'MODEL: {model_name}')

            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=0)
            y_train_pred = model.predict_proba(X_train)
            y_val_pred = model.predict_proba(X_val)
            y_pred = model.predict_proba(X_pred)

            val_preds[model_name].append(y_val_pred)
            pred_preds[model_name] = pred_preds[model_name] + y_pred

            train_logloss, train_mapk = get_metrics(
                y_train.get() if cuda.is_available() else y_train, # GPU -> CPU
                y_train_pred
            )
            val_logloss, val_mapk = get_metrics(
                y_val.get() if cuda.is_available() else y_val, # GPU -> CPU
                y_val_pred
            )

            print(f'LOGLOSS. Train: {round(train_logloss, 4)} | Val: {round(val_logloss, 4)} | Val to Train: {round(val_logloss/train_logloss, 4)}')
            print(f'MAPK.    Train: {round(train_mapk, 4)} | Val: {round(val_mapk, 4)} | Val to Train: {round(val_mapk/train_mapk, 4)}')

    pred_preds = { key: (value / n_splits) for key, value in pred_preds.items() }

    return X_folds, y_folds, val_preds, pred_preds

In [ ]:
base_params = dict(
  n_estimators=7000,
  num_class=7,
  objective='multi:softprob',
  eval_metric='mlogloss',
  booster='gbtree',
  tree_method='hist',
  device='cuda' if cuda.is_available() else 'cpu',
  n_jobs=-1,
  early_stopping_rounds=100,
  random_state=RANDOM_STATE,
)

params = dict(
    xgb01=dict(
        max_depth=12,
        learning_rate=0.02408978898626546,
        subsample=0.8722619176156262,
        colsample_bytree=0.44757526648072804,
        reg_alpha=2.8463847461990315,
        reg_lambda=0.6685176738995261,
        min_child_weight=10,
    ),
    xgb02=dict(
        max_depth=11,
        learning_rate=0.013600211117010777,
        subsample=0.8472484556777177,
        colsample_bytree=0.4590701270890474,
        reg_alpha=3.3171481083210232,
        reg_lambda=0.9381236363848697,
        min_child_weight=5,
    ),
    xgb03=dict(
        max_depth=5,
        learning_rate=0.07505389046563149,
        subsample=0.5259762849436469,
        colsample_bytree=0.4524229658909202,
        reg_alpha=2.651653141498327,
        reg_lambda=4.700559599130367,
        min_child_weight=8,
    ),
    xgb04=dict(
        max_depth=8,
        learning_rate=0.027346170065478962,
        subsample=0.9379502039625413,
        colsample_bytree=0.4095558867111752,
        reg_alpha=1.0357095290998455,
        reg_lambda=1.7172337694124145,
        min_child_weight=5,
    ),
    xgb05=dict(
        max_depth=12,
        colsample_bytree=0.467,
        subsample=0.86,
        learning_rate=0.03,
        gamma=0.26,
        max_delta_step=4,
        reg_alpha= 2.7,
        reg_lambda= 1.4,
    ),
)


for key in params.keys():
  params[key].update(base_params)


models = { key: XGBClassifier(**value) for key, value in params.items() }

In [ ]:
res = cross_validation_stacking(
    models=models,
    X=X,
    y=y,
    X_pred=X_pred,
    X_original=X_original,
    y_original=y_original
)

In [ ]:
def create_meta_dataset(cv_data):

    X_train_fold = np.asarray(cv_data[0]).reshape(-1, 8)
    y_train_fold = np.asarray(cv_data[1]).reshape(-1, 1)
    X_pred_fold = None

    for model_name, model_pred in cv_data[2].items():
        X_train_fold = np.hstack((X_train_fold, np.asarray(model_pred).reshape(-1, 7)))

        if X_pred_fold is None:
          X_pred_fold = cv_data[3][model_name]
        else:
          X_pred_fold = np.hstack((X_pred_fold, cv_data[3][model_name]))

    return X_train_fold, y_train_fold, X_pred_fold


X_train_meta, y_train_meta, X_pred_meta = create_meta_dataset(res)

In [ ]:
# to_save = pd.DataFrame(X_train_meta)
# to_save.to_csv(url_start+'/X_train_meta.csv', index=False)

# to_save = pd.DataFrame(y_train_meta).astype(np.int64).replace(label_encoder_values_inverse['Fertilizer Name'])
# to_save.to_csv(url_start+'/y_train_meta.csv', index=False)

# to_save = pd.DataFrame(X_test_meta)
# to_save.to_csv(url_start+'/X_test_meta.csv', index=False)

# optuna. meta-model hyperparameters selection

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_meta[:, 8:],
                                                    y_train_meta,
                                                    shuffle=True,
                                                    stratify=y_train_meta,
                                                    train_size=0.4)

In [ ]:
def objective(trial, X, y, shuffle=True):
    params = dict(
        max_depth=trial.suggest_int("max_depth", 2, 5),
        min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
        learning_rate=trial.suggest_float("learning_rate", 0.001, 0.1),
        subsample=trial.suggest_float("subsample", 0.7, 1.0),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.4, 1.0),
        gamma=trial.suggest_float('gamma', 0.01, 1.0, log=True),
        reg_alpha=trial.suggest_float("reg_alpha", 0.0, 7.0),
        reg_lambda=trial.suggest_float("reg_lambda", 0.0, 7.0),
        # --
        n_estimators=7000,
        num_class=7,
        objective='multi:softprob',
        eval_metric='mlogloss',
        booster='gbtree',
        tree_method='hist',
        device='cuda' if cuda.is_available() else 'cpu',
        n_jobs=-1,
        early_stopping_rounds=100,
        random_state=RANDOM_STATE,
    )

    X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=shuffle, stratify=y, test_size=0.3)

    if cuda.is_available(): # CPU -> GPU
      X_train, y_train = cp.asarray(X_train), cp.asarray(y_train)
      X_val, y_val = cp.asarray(X_val), cp.asarray(y_val)

    model = XGBClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=0)
    y_val_proba = model.predict_proba(X_val)

    if cuda.is_available(): # GPU -> CPU
      y_val = y_val.get()

    val_logloss, val_mapk = get_metrics(y_val, y_val_proba)

    trial.set_user_attr("mapk", val_mapk)

    return val_logloss


study = optuna.create_study(directions=['minimize'], sampler=optuna.samplers.TPESampler())
study.optimize(
    lambda trial: objective(trial, X_train, y_train),
    n_trials=50,
    n_jobs=-1,
    show_progress_bar=True
)


study_result = study.trials_dataframe()
study_result_dict = { trial.number: trial.params for trial in study.trials}

In [ ]:
study_result.sort_values(by='value').head(5)

# final model. submission

In [ ]:
def cross_validation(X, y, params, X_pred=None, shuffle=True, n_splits=5, random_state=RANDOM_STATE):

  y_pred = None
  kf = StratifiedKFold(n_splits=n_splits, shuffle=shuffle)

  if X_pred is not None:
      y_pred = np.zeros(( X_pred.shape[0], len(np.unique(y)) ))

      if cuda.is_available():
          X_pred = cp.asarray(X_pred)


  for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
      X_train, X_val = X[train_idx], X[val_idx]
      y_train, y_val = y[train_idx], y[val_idx]

      if cuda.is_available():
          X_train, y_train = cp.asarray(X_train), cp.asarray(y_train)
          X_val, y_val = cp.asarray(X_val), cp.asarray(y_val)

      model = XGBClassifier(**params)
      model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=500)

      y_train_pred = model.predict_proba(X_train)
      y_val_pred = model.predict_proba(X_val)

      if cuda.is_available():
          y_train = y_train.get()
          y_val = y_val.get()

      train_logloss, train_mapk = get_metrics(y_train, y_train_pred)
      val_logloss, val_mapk = get_metrics(y_val, y_val_pred)

      if X_pred is not None and y_pred is not None:
          y_pred = y_pred + model.predict_proba(X_pred)

      print(f'LOGLOSS. Train: {round(train_logloss, 4)} | Val: {round(val_logloss, 4)}. V/T: {round(val_logloss/train_logloss, 4)}')
      print(f'MAPK@3. Train: {round(train_mapk, 4)} | Val: {round(val_mapk, 4)}. V/T: {round(val_mapk/train_mapk, 4)}')

  return (y_pred / n_splits) if y_pred is not None else None

In [ ]:
res = cross_validation(
    X=X_train_meta[:, 8:],
    y=y_train_meta,
    X_pred=X_pred_meta,
    params={
        'max_depth': 3,
        'min_child_weight': 1,
        'learning_rate': 0.09463613506433619,
        'subsample': 0.8783317032856857,
        'colsample_bytree': 0.8446428679204586,
        'gamma': 0.055331104537748886,
        'reg_alpha': 5.100771478049894,
        'reg_lambda': 3.516581806624857,
        'n_estimators': 7000,
        'num_class': 7,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'booster': 'gbtree',
        'tree_method': 'hist',
        'device': 'cuda' if cuda.is_available() else 'cpu',
        'n_jobs': -1,
        'early_stopping_rounds': 300,
        'random_state': RANDOM_STATE,
    }
)

In [ ]:
y_pred = y_pred.join(pd.DataFrame(np.argsort(res)[:, -3:][:, ::-1]))
y_pred[y_pred.columns[1:]] = y_pred[y_pred.columns[1:]].replace(label_encoder_values_inverse['Fertilizer Name'])
y_pred['Fertilizer Name'] = y_pred[0] + ' ' + y_pred[1] + ' ' + y_pred[2]
y_pred.drop(columns=[0, 1, 2], inplace=True)

y_pred.head()

In [ ]:
# y_pred.to_csv('submission.csv', index=False)